## Imports ##

In [55]:
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [56]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [57]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (20)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=32) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=32) # test -> 50% validation, 50% final test

In [58]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [59]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [60]:
batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model Training ##

In [61]:
import torch.nn as nn
from src.utils import OurModel


In [62]:
ml = OurModel()

In [63]:
# model configuration
config = nn.Sequential(
    nn.Linear(20, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.Tanh(),
    nn.Linear(32, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
)
ml.add_configuration(config)

In [64]:
#GPU optimalization TODO: Doesnt work
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
ml.to(device)

cuda


In [65]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [66]:
# # Model Training
criterion = nn.BCEWithLogitsLoss()

epochs = 20
for epoch in range(epochs):
    ml.train()  # set model to training mode
    total_loss = 0
    correct = 0
    total = 0

    for inputs, true in train_loader:
        inputs, true = inputs.to(device), true.to(device)
        true = true.unsqueeze(1)
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.sigmoid(y_pred)              # convert logits → probabilities
        predicted = (preds > 0.5).float()          # threshold to 0/1
        correct += (predicted == true).sum().item()
        total += true.size(0)
    # Epoch summary
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")


In [67]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(ml.parameters(), lr=1e-4)

for epoch in range(20):
    total_loss = 0
    for inputs, true in train_loader:
        inputs, true = inputs.to(device), true.to(device)
        true = true.unsqueeze(1)
        inputs = inputs.float()
        true = true.float()
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        if torch.isnan(loss):
            print("NaN detected -> breaking")
            break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/20] | Loss: {total_loss/len(train_loader):.4f}")


## Sweep ##

In [68]:
import wandb
wandb.login()

True

In [69]:
sweep_config = {
    "method": "random",  # or "grid", "bayes"
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 0.0001, "max": 0.01},
        "epochs": {"min": 2, "max": 20},
        "hidden_size_1": {"values": [8, 16, 32, 64]},
        "hidden_size_2": {"values": [8, 16, 32, 64]},
        "hidden_size_3": {"values": [8, 16, 32, 64]},
        "n_layers": {"values": [1, 2, 3]},
        "dropout": {"min": 0.0, "max": 0.5},
        "optimizer": {"values": ["Adam", "SGD"]},
        "batch_size": {"values": [32, 64, 128, 512]}
    }
}

In [70]:
    # sw_nn = OurModel(20, 1, n_layers=3, hidden_size=[8,4,6], dropout=0)
    # sw_nn.sweeping_build(20, 1, n_layers=3, hidden_size=[8,4,6], dropout=0)


In [71]:
def train():
    wandb.init()
    CONFIG = wandb.config
    train_loader = DataLoader(train_dataset, batch_size=CONFIG.batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=CONFIG.batch_size, shuffle=False)
    # test_loader = DataLoader(test_dataset, batch_size=CONFIG.batch_size, shuffle=False)


    hl_sizes = [CONFIG.hidden_size_1, CONFIG.hidden_size_2, CONFIG.hidden_size_3]
    #Building the neural network from CONFIG
    sw_nn = OurModel()
    sw_nn.sweeping_build(20, 1, n_layers=CONFIG.n_layers, hidden_size=hl_sizes, dropout=CONFIG.dropout)
    optimizer = getattr(torch.optim, CONFIG.optimizer)(sw_nn.parameters(), lr=CONFIG.learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    #GPU computing
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sw_nn.to(device)

    for epoch in range(CONFIG.epochs):
        sw_nn.train()
        total_loss = 0
        correct = 0
        total = 0
        for inputs, true in train_loader:
            inputs, true = inputs.to(device), true.to(device)
            true = true.unsqueeze(1)
            optimizer.zero_grad()
            y_pred = sw_nn(inputs)
            loss = criterion(y_pred, true)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = torch.sigmoid(y_pred)              # convert logits → probabilities
            predicted = (preds > 0.5).float()          # threshold to 0/1
            correct += (predicted == true).sum().item()
            total += true.size(0)

        avg_train_loss = total_loss / len(train_loader)
        accuracy = correct / total

        sw_nn.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, true in val_loader:
                inputs, true = inputs.to(device), true.to(device)
                preds = sw_nn(inputs)
                true = true.unsqueeze(1)
                loss = criterion(preds, true)
                val_loss += loss.item()
                predicted = (preds > 0.5).float()
                correct += (predicted == true).sum().item()
                total += true.size(0)
        avg_val_loss = val_loss / len(val_loader)
        accuracy = correct / total

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": accuracy
        })

    # for epoch in range(5):
    #         running_loss = 0.0
    #         for Xb, yb in train_loader:
    #             optimizer.zero_grad()
    #             preds = sw_nn(Xb)
    #             loss = criterion(preds, yb)
    #             loss.backward()
    #             optimizer.step()
    #             running_loss += loss.item()
    #
    #         avg_loss = running_loss / len(train_loader)
    #         wandb.log({"epoch": epoch+1, "loss": avg_loss})


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="xbukovinam-faculty-of-informatics-and-infromation-techno")
wandb.agent(sweep_id, function=train, count=10)

Create sweep with ID: alzu00of
Sweep URL: https://wandb.ai/xbukovinam-faculty-of-informatics-and-infromation-techno/xbukovinam-faculty-of-informatics-and-infromation-techno/sweeps/alzu00of


wandb: Agent Starting Run: rq42cqi5 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.10093865643566784
wandb: 	epochs: 12
wandb: 	hidden_size_1: 64
wandb: 	hidden_size_2: 8
wandb: 	hidden_size_3: 64
wandb: 	learning_rate: 0.0030728817992451128
wandb: 	n_layers: 1
wandb: 	optimizer: Adam


epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss,█▄▃▂▂▂▂▂▁▁▁▁
val_accuracy,▃▅▁▅▃▄▃▄▇▃▅█
val_loss,▆█▇▂▅▄▂▁▁▁▁▃
epoch,12
train_loss,0.63553
val_accuracy,0.606
val_loss,0.63266


wandb: Agent Starting Run: yebscxyg with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.4692919667280961
wandb: 	epochs: 18
wandb: 	hidden_size_1: 16
wandb: 	hidden_size_2: 16
wandb: 	hidden_size_3: 16
wandb: 	learning_rate: 0.007832692196960736
wandb: 	n_layers: 1
wandb: 	optimizer: Adam


epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
train_loss,█▃▃▂▂▂▂▂▁▁▂▂▂▁▁▁▁▁
val_accuracy,█▄▂▁▇▂▇▆▅▃▄█▆▄▄▃▃▆
val_loss,█▂▂▅▄▂▅▃▂▁▁▆▂▁▁▁▁▄
epoch,18
train_loss,0.63669
val_accuracy,0.61226
val_loss,0.64086


wandb: Agent Starting Run: pi0mufc7 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.15000876467922736
wandb: 	epochs: 20
wandb: 	hidden_size_1: 32
wandb: 	hidden_size_2: 16
wandb: 	hidden_size_3: 32
wandb: 	learning_rate: 0.00989140110170397
wandb: 	n_layers: 3
wandb: 	optimizer: SGD


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,██▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▁▁▂█▂▂▂▂▆▇▃▇▂▄▂▅▂▄▅
val_loss,▄▄▃▃▄▂▂▂▄▂▂▁▅▇▂▂▁█▁▁
epoch,20
train_loss,0.64542
val_accuracy,0.58049
val_loss,0.63691
